In [1]:
!pip install --upgrade --quiet athina-client langchain langchain_community pypdf langchain-deepseek-official langchain-huggingface


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/12

In [11]:
!pip install qdrant-client langchain langchain-google-genai google-generativeai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.7/306.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6


In [17]:

# set api key
import os
from google.colab import userdata

os.environ['ATHINA_API_KEY'] = '#enteryours'

os.environ['TAVILY_API_KEY'] = '#enteryours'
os.environ['DEEPSEEK_API_KEY'] = '#enteryours'
os.environ['QDRANT_API_KEY'] = '#enteryours'


In [18]:
# load pdf
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/test.pdf")
documents = loader.load()


In [19]:
# split documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

In [20]:
# load embedding model
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5", encode_kwargs = {"normalize_embeddings": True})

## **Qdrant Vector Store**

In [21]:
# create vectorstore
from langchain_community.vectorstores import Qdrant
vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    url="#yourclusterinstance",
    prefer_grpc=True,
    collection_name="documents",
    api_key=os.environ["QDRANT_API_KEY"],
)


In [24]:
# create retriever
retriever = vectorstore.as_retriever()

## **WEB SEARCH**

In [152]:
# define web search
from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=10)

In [153]:
web_search_tool.run("Tesla stock market summary for Q3?")


[{'title': "What Drove Tesla's Q3 Earnings Beat? - Nasdaq",
  'url': 'https://www.nasdaq.com/articles/what-drove-teslas-q3-earnings-beat',
  'content': 'Tesla stock (NASDAQ:TSLA) posted a better-than-expected set of Q3 2024 earnings. Revenue for the quarter rose 8% year-over-year to $25.18 billion a year',
  'score': 0.86303437},
 {'title': 'Tesla Earnings Unexpectedly Rise 9% In Q3, Elon Musk Bullish',
  'url': 'https://www.investors.com/news/tesla-stock-earnings-q3-elon-musk/',
  'content': 'The electric-vehicle giant reported Wednesday that Q3 earnings grew 9% to 72 cents per share. Meanwhile, revenue totaled $25.18 billion, up 8%',
  'score': 0.8482825},
 {'title': 'Tesla (TSLA) Q3 2024 earnings report - CNBC',
  'url': 'https://www.cnbc.com/2024/10/23/tesla-tsla-q3-2024-earnings-report.html',
  'content': 'Best Credit Cards Best Travel Credit Cards Best Loans to Refinance Credit Card Debt Best Credit Monitoring Services Best Credit Cards for Small Business Best Personal Loans for 

## *Agentic RAG*

In [51]:
!pip install google-generativeai


In [154]:
import os
import google.generativeai as genai

os.environ["GEMINI_API"] = "#enteryours"
genai.configure(api_key=os.environ["GEMINI_API"])


In [167]:
# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=os.environ["GEMINI_API"]
)

In [168]:
# define vector search
from langchain.chains import RetrievalQA
def vector_search(query: str):
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa_chain.run(query)

In [170]:
# define web search
def web_search(query: str):
   return web_search_tool.run(query)

In [171]:
# create tool call for vector search and web search
from langchain.tools import tool

@tool
def vector_search_tool(query: str) -> str:
    return vector_search(query)

@tool
def web_search_tool_func(query: str) -> str:
    return web_search(query)

In [172]:
# define tools for the agent
from langchain.agents import Tool
tools = [
    Tool(
        name="VectorStoreSearch",
        func=vector_search_tool,
        description="Use this to search the vector store for information."
    ),
    Tool(
        name="WebSearch",
        func=web_search_tool_func,
        description="Use this to perform a web search for information."
    ),
]

In [173]:
# define agent prompt
system_prompt = """Respond to the human as helpfully and accurately as possible. You have access to the following tools: {tools}
Always try the \"VectorStoreSearch\" tool first. Only use \"WebSearch\" if the vector store does not contain the required information.
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per
JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action.
Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation"""

In [174]:
# human prompt
human_prompt = """{input}
{agent_scratchpad}
(reminder to always respond in a JSON blob)"""


In [175]:
# create prompt template
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", human_prompt),
    ]
)

In [176]:
# tool render
from langchain.tools.render import render_text_description_and_args
prompt = prompt.partial(
    tools=render_text_description_and_args(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
)

In [177]:
# create rag chain
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)

In [165]:
# create agent
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(
    agent=chain,
    tools=tools,
    handle_parsing_errors=True,
    verbose=True
)


In [101]:
agent_executor.invoke({"input": "What is the difference between Agentic RAG vs naive RAG?"})



> Entering new AgentExecutor chain...
Action:```
{
 "action": "VectorStoreSearch",
 "action_input": "What is the difference between Agentic RAG vs naive RAG?"
}```Agentic RAG uses autonomous agents to manage retrieval, filter information, and refine responses, allowing for dynamic and adaptable information processing.  Traditional RAG systems have static workflows and limited adaptability, struggling with dynamic, multi-step reasoning and complex tasks.```json
{
 "action": "Final Answer",
 "action_input": "Agentic RAG uses autonomous agents to manage retrieval, filter information, and refine responses, making it more dynamic and adaptable.  In contrast, traditional RAG systems follow static workflows and struggle with complex reasoning and multi-step tasks due to their limited adaptability."
}```

> Finished chain.


{'input': 'What is the difference between Agentic RAG vs naive RAG?',
 'output': 'Agentic RAG uses autonomous agents to manage retrieval, filter information, and refine responses, making it more dynamic and adaptable.  In contrast, traditional RAG systems follow static workflows and struggle with complex reasoning and multi-step tasks due to their limited adaptability.'}

In [178]:
agent_executor.invoke({"input": "Difference between Gemini and Deepseek?"})



> Entering new AgentExecutor chain...
Action:```
{
 "action": "VectorStoreSearch",
 "action_input": "Difference between Gemini and Deepseek"
}
```I am sorry, but I cannot provide a comparison between Gemini and Deepseek. I do not have the context to answer your question.Action:```{
 "action": "WebSearch",
 "action_input": "Difference between Gemini and Deepseek"
}
```[{'title': 'ChatGPT vs. Gemini vs. DeepSeek Understanding the Key Differences', 'url': 'https://duke-lavlesh.medium.com/chatgpt-vs-gemini-vs-deepseek-understanding-the-key-differences-b8a0c2ece25a', 'content': 'Gemini: Highly customizable for multimodal applications but may require specialized expertise. DeepSeek: Designed for easy integration into', 'score': 0.92392945}, {'title': "I tried out DeepSeek, but I'm sticking with Gemini for now", 'url': 'https://www.androidauthority.com/deepseek-vs-gemini-3521178/', 'content': 'Gemini broke things down into high-end, mid-range, and foldable models, while DeepSeek suggested t

{'input': 'Difference between Gemini and Deepseek?',
 'output': "DeepSeek and Gemini are both AI models.  Gemini is developed by Google and offers both free and paid versions (Gemini Advanced).  DeepSeek is also free to use.  Reviews comparing the two models often highlight Gemini's stronger real-time web search capabilities and overall accuracy.  While DeepSeek is a newer model and shows promise, Gemini is frequently chosen as the preferred option in direct comparisons."}

## Multiple Query Data

In [95]:
# create agent with verbose=False for production
agent_output = AgentExecutor(
    agent=chain,
    tools=tools,
    handle_parsing_errors=True,
    verbose=False
)

In [96]:
# Create dataset
question = [
    "Difference between Gemini and Deepseek?",
    "What is the difference between Agentic RAG vs naive RAG?"
]
response = []
contexts = []

# Inference
for query in question:
    vector_contexts = retriever.get_relevant_documents(query)
    if vector_contexts:
        context_texts = [doc.page_content for doc in vector_contexts]
        contexts.append(context_texts)
    else:
        print(f"[DEBUG] No relevant information in vector store for query: {query}. Falling back to web search.")
        web_results = web_search_tool.run(query)
        contexts.append([web_results])

    # Get the agent response
    result = agent_output.invoke({"input": query})
    response.append(result['output'])

<ipython-input-96-754372c8a142>:11: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  vector_contexts = retriever.get_relevant_documents(query)


In [97]:

# To dict
data = {
    "query": question,
    "response": response,
    "context": contexts,
}

# Connecting to Athina IDE

In [98]:
# Format the data for Athina
rows = []
for i in range(len(data["query"])):
    row = {
        'query': data["query"][i],
        'context': data["context"][i],
        'response': data["response"][i],
    }
    rows.append(row)

In [100]:
#connect to Athina
from athina_client.datasets import Dataset
from athina_client.keys import AthinaApiKey

AthinaApiKey.set_key(os.environ['ATHINA_API_KEY'])

try:
    Dataset.add_rows(
        dataset_id='7fa5efad-4ac5-40d1-960c-40183dc52a29',
        rows=rows
    )
except Exception as e:
    print(f"Failed to add rows: {e}")